In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
import re
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("../finalRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows

ar = em.read_csv_metadata("../finalArkives.csv", encoding="ISO-8859-1", key='id')


c:\users\aparn\anaconda\envs\my_first_env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [2]:
rl['block'] = 'yes'
ar['block'] = 'yes'

In [3]:

rl.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)
ar.rename(columns={'Unnamed: 0':'Unnamed'}, inplace=True)

In [4]:
ar.head()

,Unnamed,id,scientific_name,name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threat_keywords,conservation_keywords,tCount,cCount,block
0,0,0,Papasula abbotti,abbotts booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula,Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0.0,yes
1,1,1,Cephalophus spadix,abbotts duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50 â?? 60 kg (2),Like many other duikers Abbottâ??s duiker is being impacted by a combination of hunting and habi...,Although Abbottâ??s duiker occurs within several protected areas such as the Kilimanjaro Nationa...,hunting;,protected;,1,1.0,yes
2,2,2,Cephalophus spadix,abbotts duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus,50 â?? 60 kg (2),Like many other duikers Abbottâ??s duiker is being impacted by a combination of hunting and habi...,Although Abbottâ??s duiker occurs within several protected areas such as the Kilimanjaro Nationa...,hunting;,protected;,1,1.0,yes
3,3,3,Tiliqua adelaidensis,adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua,Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australiaâ??s approximate...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1.0,yes
4,4,4,Spheniscus demersus,african penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus,Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2.0,yes


In [5]:
#Block on bogus attribute so that we can actually do
#blocking on genus
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'block', 'block', 
                     l_output_attrs=list(rl.columns.values), 
                     r_output_attrs=list(ar.columns.values))
len(C0)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:49


12687745

In [6]:
C1 = ob.block_candset(C0, 'genus', 'genus')
len(C1)

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:03:25


3850

In [7]:
feat = em.get_features_for_matching(rl, ar)

In [8]:
H = em.extract_feature_vecs(C1, 
                            feature_table=feat, 
                            attrs_after='rtable_block',
                            show_progress=False) 

In [9]:
#em.to_csv_metadata(C1, './candidate_set.csv')
#C1 = C1.drop('rtable_block', 1)
#C1 = C1.drop('ltable_block', 1)
result = pd.concat([C1, H], axis=1)
#result = result.drop('rtable_block',1)
result.columns


Index(['_id', 'ltable_id', 'rtable_id', 'ltable_Unnamed', 'ltable_name',
       'ltable_genus', 'ltable_family', 'ltable_ecology', 'ltable_countries',
       'ltable_threat_paragraph', 'ltable_conservation_paragraph',
       'ltable_pop_trend', 'ltable_status', 'ltable_country_count',
       'ltable_scientific_name', 'ltable_block', 'rtable_Unnamed',
       'rtable_scientific_name', 'rtable_name', 'rtable_kingdom',
       'rtable_phylum', 'rtable_class', 'rtable_order', 'rtable_family',
       'rtable_genus', 'rtable_size', 'rtable_threats', 'rtable_conservation',
       'rtable_threat_keywords', 'rtable_conservation_keywords',
       'rtable_tCount', 'rtable_cCount', 'rtable_block', '_id', 'ltable_id',
       'rtable_id', 'Unnamed_Unnamed_exm', 'Unnamed_Unnamed_anm',
       'Unnamed_Unnamed_lev_dist', 'Unnamed_Unnamed_lev_sim',
       'name_name_jac_qgm_3_qgm_3', 'name_name_cos_dlm_dc0_dlm_dc0',
       'name_name_jac_dlm_dc0_dlm_dc0', 'name_name_mel', 'name_name_lev_dist',
       'nam

In [10]:
result

,_id,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,...,scientific_name_scientific_name_lev_sim,scientific_name_scientific_name_nmw,scientific_name_scientific_name_sw,block_block_lev_dist,block_block_lev_sim,block_block_jar,block_block_jwn,block_block_exm,block_block_jac_qgm_3_qgm_3,rtable_block
25215,25215,1470,3,1470,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,...,1.000000,20.0,20.0,0.0,1.0,1.0,1.0,1,1.0,yes
75598,75598,4363,9,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,...,1.000000,22.0,22.0,0.0,1.0,1.0,1.0,1,1.0,yes
75600,75600,4365,9,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,...,0.608696,13.0,13.0,0.0,1.0,1.0,1.0,1,1.0,yes
78778,78778,7543,9,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,...,0.636364,13.0,13.0,0.0,1.0,1.0,1.0,1,1.0,yes
83513,83513,4363,10,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\r\r\n The major threat is drainage of wetland habitats and aquatic pollution of many waterwa...,...,1.000000,22.0,22.0,0.0,1.0,1.0,1.0,1,1.0,yes
83515,83515,4365,10,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\r\r\n The major threat is over-collecting for food; other threats include drought and water ...,...,0.608696,13.0,13.0,0.0,1.0,1.0,1.0,1,1.0,yes
86693,86693,7543,10,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\r\r\n The major threat is habitat loss and degradation of suitable aquatic habitats. Althoug...,...,0.636364,13.0,13.0,0.0,1.0,1.0,1.0,1,1.0,yes
105744,105744,2849,13,2849,"alexandria cycad, dune cycad",Encephalartos,Zamiaceae,Terrestrial,South Africa (Eastern Cape Province);,\r\r\n The major threat to this species is removal by collectors. This has been the main caus...,...,1.000000,23.0,23.0,0.0,1.0,1.0,1.0,1,1.0,yes
105745,105745,2850,13,2850,chimanimani cycad,Encephalartos,Zamiaceae,Terrestrial,Mozambique;,\r\r\n This species is threatened due to over-collecting for ornamental purposes.\r\r\n\r\r\n...,...,0.600000,11.0,16.0,0.0,1.0,1.0,1.0,1,1.0,yes
105746,105746,2851,13,2851,NaN,Encephalartos,Zamiaceae,Terrestrial,"Tanzania, United Republic of;","\r\r\n This species is affected by too frequent fires, which may affect regeneration. Changes...",...,0.695652,16.0,16.0,0.0,1.0,1.0,1.0,1,1.0,yes


In [11]:
label = em.split_train_test(C1, train_proportion=0.1039)
gold = label['train']
DS = label['test']


#Automatically label the training data
labels = []
matches = 0;
nonmatches = 0;
for index, row in gold.iterrows():
    if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
        labels.append(1)
        matches = matches + 1
    else:
        labels.append(0)
        nonmatches = nonmatches + 1
        
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

number of matches in training set: 37
number of nonmatches in training set: 363


In [12]:
gold


,_id,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,...,rtable_genus,rtable_size,rtable_threats,rtable_conservation,rtable_threat_keywords,rtable_conservation_keywords,rtable_tCount,rtable_cCount,rtable_block,label
4865605,4865605,5795,614,5795,NaN,Melanopsis,Melanopsidae,Freshwater,Syrian Arab Republic;,\r\r\n The part of the Syrian coast where this species occurs has been heavily urbanized and ...,...,Melanopsis,NaN,Removal of water from the sites in which <em>Melanopsis etrusca</em> lives is a major threat to ...,<em>Melanopsis etrusca </em>is a protected species in Tuscany and is listed under Annexes A and ...,invasive;pet;,protected;,2,1.0,yes,0
2111635,2111635,6245,266,6245,NaN,Shorea,Dipterocarpaceae,Terrestrial,Malaysia (Sarawak);,\r\r\n Habitat destruction.\r\r\n\r\r\n \r\r\n,...,Shorea,Trunk diameter: up to 172 cm (2),Dipterocarp forests have become amongst the most endangered in the world widely logged for use i...,Some subpopulations of this species are found in primary forest reserves where they receive vary...,,,0,0.0,yes,0
8349249,8349249,6839,1054,6839,NaN,Mammillaria,Cactaceae,Terrestrial,Mexico (Hidalgo);,"\r\r\n The known threat for this species is illegal collecting, which was thought to drive th...",...,Mammillaria,Stem diameter: 25 cm (2),This highly attractive and unusual cactus is extremely popular with collectors and the location ...,In recognition of the worrying population decline the Biznaguita was upgraded to Critically Enda...,,cites;,0,1.0,yes,0
1563827,1563827,4572,197,4572,twofingered skink,Chalcides,Scincidae,Terrestrial,Spain; Algeria; Morocco;,\r\r\n Development of coastal areas for tourism and military purposes are major threats to th...,...,Chalcides,Chalcides (1),Perhaps the most threatened skink species in the Canaries (5) <em>Chalcides simonyi</em> has a s...,Although <em>Chalcides simonyi</em> has not been the target of any known conservation measures i...,loss;,protected;,1,1.0,yes,0
12496270,12496270,6400,1578,6400,yellow meranti,Shorea,Dipterocarpaceae,Terrestrial,Singapore;,NaN,...,Shorea,Height: up to 50 m (2)Trunk diameter: up to 182 cm (2),Numbers of white lauan have become depleted due to logging and <strong>slash-and-burn</strong> a...,The white lauan most likely occurs in a number of protected areas throughout the Philippines suc...,,protected;,0,1.0,yes,0
8982479,8982479,6869,1134,6869,venda cycad,Encephalartos,Zamiaceae,Terrestrial,South Africa (Limpopo Province);,\r\r\n This species is threatened due to over-collecting for ornamental purposes.\r\r\n\r\r\n...,...,Encephalartos,Trunk length: up to 4.2 m (2)Trunk diameter: 35 - 40 cm (2),Over the past few decades many South African cycads have become increasingly scarce in the wild ...,There are not known to be any specific conservation measures in place for this Critically Endang...,loss;,cites;protected;,1,2.0,yes,1
10992717,10992717,6697,1388,6697,NaN,Ravenea,Palmae,Freshwater,Madagascar;,\r\r\n There are a number of threats to this species including habitat clearance along the ri...,...,Ravenea,Trunk height: 2.5 - 8 m (2),This palm is only known from a single river and is therefore extremely vulnerable to chance even...,Should this unique species be tragically lost from the wild there are hopes that it will persist...,,,0,0.0,yes,1
4377693,4377693,698,553,698,terredehaut racer,Alsophis,Dipsadidae,Terrestrial,Guadeloupe;,"\r\r\n This species is threatened by habitat loss due to urban development, particularly for ...",...,Alsophis,Length: 110 cm (2),The Leeward Island racer population is declining and has become extinct in some areas. This is d...,One of the rarest snakes of the Lesser Antilles the Leeward Island racer will only survive if in...,,,0,0.0,yes,0
11158354,11158354,6119,1409,6119,white meranti,Shorea,Dipterocarpaceae,Terrestrial,"Indonesia (Sumatera); Malaysia (Peninsular Malaysia, Sarawak);",NaN,...,Shorea,Height: 70 m (2)Trunk 

In [13]:
em.to_csv_metadata(gold, './gold.csv')
em.to_csv_metadata(DS, './unlabelled.csv')

True

In [14]:
test_train = em.split_train_test(gold, train_proportion=0.7)
train = test_train['train']
test = test_train['test']
len(train)

280

In [15]:
em.to_csv_metadata(gold, './train.csv')
em.to_csv_metadata(DS, './test.csv')

File already exists at ./train.csv; Overwriting it
Metadata file already exists at ./train.metadata. Overwriting it
File already exists at ./test.csv; Overwriting it
Metadata file already exists at ./test.metadata. Overwriting it


True